In [1]:
import numpy as np
import glob
import os
import netCDF4 as nc4
import pickle as pkl
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import matplotlib.tri as tri
import math
from scipy.interpolate import interp1d
import sim_name_remap as map
import var_config as vc

In [2]:
def get_struc_var(FILES, var, prs_lvls, trp_lvls, aer_model='M'):

    file0 = nc4.Dataset(FILES[0])        
    hyam = file0.variables['hyam'][:] 
    hybm = file0.variables['hybm'][:]    
    lon = file0.variables['lon'][:]
    lat = file0.variables['lat'][:]
    
    if aer_model == 'M': var_strs = vc.var_info[var]['MAM4_vars']
    if aer_model == 'C': var_strs = vc.var_info[var]['CARMA_vars']

    var_array_plev   = np.zeros((1,len(prs_lvls),len(lat),len(lon)))-999
    var_array_trplev = np.zeros((1,len(trp_lvls),len(lat),len(lon)))-999
    date_array = []
    datesec_array = []

    for f in range(len(FILES)):

        if f % 10 == 0: print('*** File ', f+1, ' of ', len(FILES))

        file = nc4.Dataset(FILES[f])
        
        if len(var_strs) > 1:        
            var_array = []
            for sub_var in var_strs:
                var_array.append(file.variables[sub_var][:])
            var_array = np.asarray(var_array)
            var_array = np.nansum(var_array, axis=0)
        # If not, read the variable directly
        else:
            var_array = file.variables[var_strs[0]][:] 
        
        PS = file.variables['PS'][:]
        #### TROP_P = file.variables['TROP_P'][:]  Everything with 4 #s will allow the trop rel capability to run again
        date = file.variables['date'][:]
        datesec = file.variables['datesec'][:]
        #GPH = file.variables['Z3'][:]

        NT = np.shape(var_array)[0]

        curr_var_array_plev = np.zeros((NT,len(prs_lvls),len(lat),len(lon)))-999
        curr_var_array_trplev = np.zeros((NT,len(trp_lvls),len(lat),len(lon)))-999

        for t in range(NT):
            
            date_array.append(date[t])
            datesec_array.append(datesec[t])
            
            for ln in range(len(lon)):
                for lt in range(len(lat)):

                    curr_PS = PS[t,lt,ln]           
                    curr_plev = (hyam*1.e5 + hybm*curr_PS)/100.
                    #plev_f = interp1d(curr_plev, np.arange(len(curr_plev)), bounds_error=False, fill_value=-999)

                    curr_varprof = np.squeeze(var_array[t,:,lt,ln])
                    #### curr_TROP_P = TROP_P[t,lt,ln]/100.
                    #curr_gphprof = np.squeeze(GPH[t,:,lt,ln])
                
                    # Fill the pressure level array
                    
                    pvar_f = interp1d(curr_plev, np.squeeze(curr_varprof), bounds_error = False, fill_value = -999)
                    curr_var_array_plev[t,:,lt,ln] = pvar_f(prs_lvls)
    
                    # Fill the tropopause-relative array 
                    
                    #trop_index_frac = plev_f(curr_TROP_P)
                    #### curr_deltaplev = curr_plev - curr_TROP_P

                    #### f_deltap = interp1d(curr_deltaplev, curr_varprof, bounds_error=False, fill_value=-999) 
                    
                    #### curr_var_array_trplev[t,:,lt,ln] = f_deltap(trp_lvls)
                
                    #gph_f = interp1d(np.arange(len(curr_gphprof)), curr_gphprof, bounds_error=False, fill_value=-999)
                    #curr_TROP_gph = gph_f(trop_index_frac)
    
                    #curr_deltazlev = (curr_gphprof - curr_TROP_gph)/1.0e3
                    #f_deltaz = interp1d(curr_deltazlev, curr_varprof, bounds_error=False, fill_value=-999)   
                                    
        var_array_plev   = np.append(var_array_plev, curr_var_array_plev, axis=0)
        #### var_array_trplev = np.append(var_array_trplev, curr_var_array_trplev, axis=0)     

    return var_array_plev[1:,:,:,:], -999, lon, lat, date_array, datesec_array   #### var_array_trplev[1:,:,:,:]



In [3]:
vars = ['SO4']    # 'aer_num','BC','OA'  'CO','SO2',
sims = ['MAM4_MEIC_SO2_rad','MAM4_CAMS6.2_SO2_rad'] # Pick the _rad simulation so we can average all the h0 files from 2010-2020     'CARMA_CAMS_SO2_rad','CARMA_MEIC_SO2_rad',

prs_lvls = [70.,100.,150.,200.,500.,700.,850.]
trp_lvls = []     
SAVEDIR = '/glade/derecho/scratch/wsmith/carma_vars/map_vars/'


In [4]:
for sim in sims: 

    sim_path = map.remap[sim]
    FILES = sorted(glob.glob(sim_path))

    for v in range(len(vars)):
    
        var = vars[v]
        print(var)
        
        var_array_plev, var_array_trplev, lon, lat, date, datesec = get_struc_var(FILES, var, prs_lvls, trp_lvls, aer_model = sim[0])
    
        with open(SAVEDIR + sim + '_' + var + '_MapData_plev.pkl', 'wb') as f:  
            pkl.dump((var_array_plev, prs_lvls, lon, lat, date, datesec), f)  
    
        #with open(SAVEDIR + sim + '_' + var + '_MapData_trplev.pkl', 'wb') as f:  
        #    pkl.dump((var_array_trplev, trp_lvls, lon, lat, date, datesec), f)

SO4
*** File  1  of  132
*** File  11  of  132
*** File  21  of  132
*** File  31  of  132
*** File  41  of  132
*** File  51  of  132
*** File  61  of  132
*** File  71  of  132
*** File  81  of  132
*** File  91  of  132
*** File  101  of  132
*** File  111  of  132
*** File  121  of  132
*** File  131  of  132
SO4
*** File  1  of  132
*** File  11  of  132
*** File  21  of  132
*** File  31  of  132
*** File  41  of  132
*** File  51  of  132
*** File  61  of  132
*** File  71  of  132
*** File  81  of  132
*** File  91  of  132
*** File  101  of  132
*** File  111  of  132
*** File  121  of  132
*** File  131  of  132
